Weather API - OpenMeteo

* Este notebook ha sido creado para añadir información al dataframe obtenido en el paso anterior. La idea es poder determinar el tiempo que hacía en el puerto de llegada del buque. 

In [ ]:
pip install openmeteo-requests

In [ ]:
pip install retry-requests

In [ ]:
pip install requests-cache

In [14]:
import pandas as pd
import openmeteo_requests
import requests_cache
from retry_requests import retry

ModuleNotFoundError: No module named 'openmeteo'

In [2]:
#Voy a importar en primer lugar el dataframe
df = pd.read_csv('../data/transit_clean.csv')
df.head()

,barco,origen,destino,llegada,salida,alineacion,consignatorio,gt,escala,bandera,eslora,calado
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VALIANT LADY,CANNES,IBIZA,31/05/2023 17:44,01/06/2023 18:15,ALIN.NORTE DE LA PLATAFORMA (del 22 al 2),ESTELA SHIPPING PALMA SA,108.192,192.0,BAHAMAS,277.0,8.0
2,ELEANOR ROOSEVELT,IBIZA,IBIZA,31/05/2023 22:28,01/06/2023 07:51,MUELLE PARAIRES (del 11 al 2),BALEARIA EUROLINEAS MARITIMAS S.A.U.,12.262,2066.0,CHIPRE,123.0,4.0
3,TENACIA,BARCELONA,BARCELONA,01/06/2023 04:23,01/06/2023 09:59,M. 1ª ALIN. PONIENTE SUR (del 14 al 1),TRANSPORTES Y CONSIGNACIONES MARITIMAS BALEAR SAU,25.993,2462.0,ITALIA,199.0,6.0
4,HEDY LAMARR,Valencia,Valencia,01/06/2023 04:53,01/06/2023 10:21,ALIN.OESTE DE LA PLATAFORMA (del 6 al 1),BALEARIA EUROLINEAS MARITIMAS S.A.U.,26.375,2422.0,CHIPRE,186.0,7.0


In [3]:
# Borramos la primera fila del dataframe
df = df.drop(df.index[0])

In [4]:
df.head()

,barco,origen,destino,llegada,salida,alineacion,consignatorio,gt,escala,bandera,eslora,calado
1,VALIANT LADY,CANNES,IBIZA,31/05/2023 17:44,01/06/2023 18:15,ALIN.NORTE DE LA PLATAFORMA (del 22 al 2),ESTELA SHIPPING PALMA SA,108.192,192.0,BAHAMAS,277.0,8.0
2,ELEANOR ROOSEVELT,IBIZA,IBIZA,31/05/2023 22:28,01/06/2023 07:51,MUELLE PARAIRES (del 11 al 2),BALEARIA EUROLINEAS MARITIMAS S.A.U.,12.262,2066.0,CHIPRE,123.0,4.0
3,TENACIA,BARCELONA,BARCELONA,01/06/2023 04:23,01/06/2023 09:59,M. 1ª ALIN. PONIENTE SUR (del 14 al 1),TRANSPORTES Y CONSIGNACIONES MARITIMAS BALEAR SAU,25.993,2462.0,ITALIA,199.0,6.0
4,HEDY LAMARR,Valencia,Valencia,01/06/2023 04:53,01/06/2023 10:21,ALIN.OESTE DE LA PLATAFORMA (del 6 al 1),BALEARIA EUROLINEAS MARITIMAS S.A.U.,26.375,2422.0,CHIPRE,186.0,7.0
5,HYPATIA DE ALEJANDRIA,BARCELONA,BARCELONA,01/06/2023 05:19,01/06/2023 11:35,MUELLE DE LA 2 ALIN. DEL DIQUE DEL OESTE (del ...,BALEARIA EUROLINEAS MARITIMAS S.A.U.,28.658,2181.0,CHIPRE,187.0,7.0


In [5]:
# Para pedirle información a la API de clima en un puerto debemos pasarle su latitud y longitud. Vamos a sacar cuantos destinos tenemos:

df.destino.unique()

array(['IBIZA', 'BARCELONA', 'Valencia', 'MAHON, MENORCA', 'SAGUNTO',
       'AJACCIO', 'LA SPEZIA', 'ALICANTE', 'GENOA', 'SALERNO', 'MALAGA',
       'CADIZ', 'MARSEILLE', 'ALTEA', 'PALERMO', 'CAGLIARI', 'ORAN',
       'A Órdenes', 'OLBIA (TERRANOVA)', 'CIVITAVECCHIA', 'ALCANAR',
       'ALCUDIA', 'PIRAEUS', 'ALMERIA', 'PIOMBINO',
       'SARROCH (PORTO FOXI)', 'CARTAGENA', 'CASTELLON', 'TOULON',
       'GIBRALTAR', 'SEVILLA', 'LIVORNO', 'FOS-SUR-MER', 'LISBOA',
       'MALTA (VALETTA)', 'VILLEFRANCHE-SUR-SAONE', 'GANDIA',
       'SAN PEDRO DEL PINATAR', 'NICE', 'SOUTHAMPTON', 'PROPRIANO',
       'MOTRIL', 'PORT EVERGLADES'], dtype=object)

* Vamos a necesitar las latitudes y longitudes de todos estos puertos. Vamos a almacenar la información en un dataframe
* Creamos un dataframe con tantas filas como puertos tengamos y con 2 columnas: latitud y longitud


In [6]:
puertos = ['IBIZA', 'BARCELONA', 'Valencia', 'MAHON, MENORCA', 'SAGUNTO',
       'AJACCIO', 'LA SPEZIA', 'ALICANTE', 'GENOA', 'SALERNO', 'MALAGA',
       'CADIZ', 'MARSEILLE', 'ALTEA', 'PALERMO', 'CAGLIARI', 'ORAN',
       'A Órdenes', 'OLBIA (TERRANOVA)', 'CIVITAVECCHIA', 'ALCANAR',
       'ALCUDIA', 'PIRAEUS', 'ALMERIA', 'PIOMBINO',
       'SARROCH (PORTO FOXI)', 'CARTAGENA', 'CASTELLON', 'TOULON',
       'GIBRALTAR', 'SEVILLA', 'LIVORNO', 'FOS-SUR-MER', 'LISBOA',
       'MALTA (VALETTA)', 'VILLEFRANCHE-SUR-SAONE', 'GANDIA',
       'SAN PEDRO DEL PINATAR', 'NICE', 'SOUTHAMPTON', 'PROPRIANO',
       'MOTRIL', 'PORT EVERGLADES'] # definimos lista de puertos

puertos_coords = pd.DataFrame() # Creamos un dataframe vacío

puertos_coords['Puerto'] = puertos # Inicializamos las filas con los nombres de cada puerto

puertos_coords['Latitud'] = 0.0 # Inicializamos las columnas de lat y long.
puertos_coords['Longitud'] = 0.0

puertos_coords.loc[puertos_coords.Puerto == 'IBIZA', ['Latitud', 'Longitud']] = [38.912653, 1.442283]
puertos_coords.loc[puertos_coords.Puerto == 'BARCELONA', ['Latitud', 'Longitud']] = [41.346387, 2.165532]
puertos_coords.loc[puertos_coords.Puerto == 'Valencia', ['Latitud', 'Longitud']] = [39.467328, -0.390288]
puertos_coords.loc[puertos_coords.Puerto == 'MAHON, MENORCA', ['Latitud', 'Longitud']] = [39.90440, 4.247600 ]
puertos_coords.loc[puertos_coords.Puerto == 'SAGUNTO', ['Latitud', 'Longitud']] = [39.783600, -0.210800]
puertos_coords.loc[puertos_coords.Puerto == 'AJACCIO', ['Latitud', 'Longitud']] = [42.058900, 8.731300]
puertos_coords.loc[puertos_coords.Puerto == 'LA SPEZIA', ['Latitud', 'Longitud']] = [43.948000, 9.683300]
puertos_coords.loc[puertos_coords.Puerto == 'ALICANTE', ['Latitud', 'Longitud']] = [38.362500, -0.478900]
puertos_coords.loc[puertos_coords.Puerto == 'GENOA', ['Latitud', 'Longitud']] = [44.406900, 8.966700]
puertos_coords.loc[puertos_coords.Puerto == 'SALERNO', ['Latitud', 'Longitud']] = [40.642900, 14.744700]
puertos_coords.loc[puertos_coords.Puerto == 'MALAGA', ['Latitud', 'Longitud']] = [36.720300, -4.420500]
puertos_coords.loc[puertos_coords.Puerto == 'CADIZ', ['Latitud', 'Longitud']] = [36.529400,	-6.251900]
puertos_coords.loc[puertos_coords.Puerto == 'MARSEILLE', ['Latitud', 'Longitud']] = [43.283300,	5.366700]
puertos_coords.loc[puertos_coords.Puerto == 'ALTEA', ['Latitud', 'Longitud']] = [38.625000,	-0.375000]
puertos_coords.loc[puertos_coords.Puerto == 'PALERMO', ['Latitud', 'Longitud']] = [38.115800, 13.351900]
puertos_coords.loc[puertos_coords.Puerto == 'CAGLIARI', ['Latitud', 'Longitud']] = [39.220800, 9.115800]
puertos_coords.loc[puertos_coords.Puerto == 'ORAN', ['Latitud', 'Longitud']] = [35.666700, -0.666700]
puertos_coords.loc[puertos_coords.Puerto == 'A Órdenes', ['Latitud', 'Longitud']] = [43.450000, -7.200000]
puertos_coords.loc[puertos_coords.Puerto == 'OLBIA (TERRANOVA)', ['Latitud', 'Longitud']] = [41.245600,	9.475000]
puertos_coords.loc[puertos_coords.Puerto == 'CIVITAVECCHIA', ['Latitud', 'Longitud']] = [42.008300, 11.750000]
puertos_coords.loc[puertos_coords.Puerto == 'ALCANAR', ['Latitud', 'Longitud']] = [40.766700, 0.216700]
puertos_coords.loc[puertos_coords.Puerto == 'ALCUDIA', ['Latitud', 'Longitud']] = [39.520800, 2.892500]
puertos_coords.loc[puertos_coords.Puerto == 'PIRAEUS', ['Latitud', 'Longitud']] = [37.983300, 23.666700]
puertos_coords.loc[puertos_coords.Puerto == 'ALMERIA', ['Latitud', 'Longitud']] = [36.833300, -2.483300]
puertos_coords.loc[puertos_coords.Puerto == 'PIOMBINO', ['Latitud', 'Longitud']] = [42.700000, 10.333300]
puertos_coords.loc[puertos_coords.Puerto == 'SARROCH (PORTO FOXI)', ['Latitud', 'Longitud']] = [39.283300, 3.666700]
puertos_coords.loc[puertos_coords.Puerto == 'CARTAGENA', ['Latitud', 'Longitud']] = [37.216700, -0.98330]
puertos_coords.loc[puertos_coords.Puerto == 'CASTELLON', ['Latitud', 'Longitud']] = [40.000000, -0.000]
puertos_coords.loc[puertos_coords.Puerto == 'TOULON', ['Latitud', 'Longitud']] = [43.133300, 5.916700]
puertos_coords.loc[puertos_coords.Puerto == 'GIBRALTAR', ['Latitud', 'Longitud']] = [36.133300, -5.33330]
puertos_coords.loc[puertos_coords.Puerto == 'SEVILLA', ['Latitud', 'Longitud']] = [37.383300, -5.96670]
puertos_coords.loc[puertos_coords.Puerto == 'LIVORNO', ['Latitud', 'Longitud']] = [43.366700, 10.366700]
puertos_coords.loc[puertos_coords.Puerto == 'FOS-SUR-MER', ['Latitud', 'Longitud']] = [43.125000, 5.25]
puertos_coords.loc[puertos_coords.Puerto == 'LISBOA', ['Latitud', 'Longitud']] = [38.722222, -9.13888]
puertos_coords.loc[puertos_coords.Puerto == 'MALTA (VALETTA)', ['Latitud', 'Longitud']] = [35.900000, 14.50]
puertos_coords.loc[puertos_coords.Puerto == 'VILLEFRANCHE-SUR-SAONE', ['Latitud', 'Longitud']] = [43.666667, 4.90]
puertos_coords.loc[puertos_coords.Puerto == 'GANDIA', ['Latitud', 'Longitud']] = [39.100000, -0.30]
puertos_coords.loc[puertos_coords.Puerto == 'SAN PEDRO DEL PINATAR', ['Latitud', 'Longitud']] = [37.500000, -0.75]
puertos_coords.loc[puertos_coords.Puerto == 'NICE', ['Latitud', 'Longitud']] = [43.700000, 7.25]
puertos_coords.loc[puertos_coords.Puerto == 'SOUTHAMPTON', ['Latitud', 'Longitud']] = [50.900000, -1.40]
puertos_coords.loc[puertos_coords.Puerto == 'PROPRIANO', ['Latitud', 'Longitud']] = [42.750000, 10.33]
puertos_coords.loc[puertos_coords.Puerto == 'MOTRIL', ['Latitud', 'Longitud']] = [36.750000, -3.75]
puertos_coords.loc[puertos_coords.Puerto == 'PORT EVERGLADES', ['Latitud', 'Longitud']] = [26.166667, -80.166667]


puertos_coords

,Puerto,Latitud,Longitud
0,IBIZA,38.912653,1.442283
1,BARCELONA,41.346387,2.165532
2,Valencia,39.467328,-0.390288
3,"MAHON, MENORCA",39.904400,4.247600
4,SAGUNTO,39.783600,-0.210800
5,AJACCIO,42.058900,8.731300
6,LA SPEZIA,43.948000,9.683300
7,ALICANTE,38.362500,-0.478900
8,GENOA,44.406900,8.966700
9,SALERNO,40.642900,14.744700


* Una vez tenemos las coordenadas de cada puerto vamos a guardar la tabla en nuestra carpeta de data y posteriormente fusionar la info en el dataframe original. 

In [7]:
puertos_coords.to_csv('../data/puertos.csv', index=False)

In [8]:
df = df.merge(puertos_coords, left_on='destino', right_on='Puerto', how='left')
df = df.rename(columns={'Latitud': 'LatitudPuertoLlegada', 'Longitud': 'LongitudPuertoLlegada'})
df.head()


,barco,origen,destino,llegada,salida,alineacion,consignatorio,gt,escala,bandera,eslora,calado,Puerto,LatitudPuertoLlegada,LongitudPuertoLlegada
0,VALIANT LADY,CANNES,IBIZA,31/05/2023 17:44,01/06/2023 18:15,ALIN.NORTE DE LA PLATAFORMA (del 22 al 2),ESTELA SHIPPING PALMA SA,108.192,192.0,BAHAMAS,277.0,8.0,IBIZA,38.912653,1.442283
1,ELEANOR ROOSEVELT,IBIZA,IBIZA,31/05/2023 22:28,01/06/2023 07:51,MUELLE PARAIRES (del 11 al 2),BALEARIA EUROLINEAS MARITIMAS S.A.U.,12.262,2066.0,CHIPRE,123.0,4.0,IBIZA,38.912653,1.442283
2,TENACIA,BARCELONA,BARCELONA,01/06/2023 04:23,01/06/2023 09:59,M. 1ª ALIN. PONIENTE SUR (del 14 al 1),TRANSPORTES Y CONSIGNACIONES MARITIMAS BALEAR SAU,25.993,2462.0,ITALIA,199.0,6.0,BARCELONA,41.346387,2.165532
3,HEDY LAMARR,Valencia,Valencia,01/06/2023 04:53,01/06/2023 10:21,ALIN.OESTE DE LA PLATAFORMA (del 6 al 1),BALEARIA EUROLINEAS MARITIMAS S.A.U.,26.375,2422.0,CHIPRE,186.0,7.0,Valencia,39.467328,-0.390288
4,HYPATIA DE ALEJANDRIA,BARCELONA,BARCELONA,01/06/2023 05:19,01/06/2023 11:35,MUELLE DE LA 2 ALIN. DEL DIQUE DEL OESTE (del ...,BALEARIA EUROLINEAS MARITIMAS S.A.U.,28.658,2181.0,CHIPRE,187.0,7.0,BARCELONA,41.346387,2.165532


In [9]:
# Guardamos esta versión del dataframe en un csv en nuestra carpeta de Data
df.to_csv('../data/tranist_coords.csv', index=False)

A continuación se va a hacer una llamada a la API Open-Meteo para crear dos columnas más en el dataframe que incluyan el total de lluvia y viento máximo medido en el día. 

Una cosa que debemos hacer antes de poder llamar a la API es cambiar el formato de las fechas en las columnas 'llegada' y 'salida' a formato YYYY-MM-DD. Hacemos eso y llamamos a la API para introducir las dos nuevas columnas.

In [10]:
# No me interesa perder la hora porque luego puede ser interesante ver como afect el clima al tiempo que un barco está atracado en un puerto.
# Voy a proceder a cambiar el formato de la fecha y a almacaenar la hora en una columna aparte

# Primero paso la columna a tipo datetime con el formato original:
df['llegada'] = pd.to_datetime(df['llegada'], format='%d/%m/%Y %H:%M')
df['salida'] = pd.to_datetime(df['salida'], format='%d/%m/%Y %H:%M')

# Y ahora vamos a pasar la fecha al formato que nos interesa y a almacenar la hora en una columna aparte para llegada y salida
df['llegada_fecha'] = df['llegada'].dt.strftime('%Y-%m-%d')
df['llegada_hora'] = df['llegada'].dt.strftime('%H:%M')
df['salida_fecha'] = df['salida'].dt.strftime('%Y-%m-%d')
df['salida_hora'] = df['salida'].dt.strftime('%H:%M')
df.head()

,barco,origen,destino,llegada,salida,alineacion,consignatorio,gt,escala,bandera,eslora,calado,Puerto,LatitudPuertoLlegada,LongitudPuertoLlegada,llegada_fecha,llegada_hora,salida_fecha,salida_hora
0,VALIANT LADY,CANNES,IBIZA,2023-05-31 17:44:00,2023-06-01 18:15:00,ALIN.NORTE DE LA PLATAFORMA (del 22 al 2),ESTELA SHIPPING PALMA SA,108.192,192.0,BAHAMAS,277.0,8.0,IBIZA,38.912653,1.442283,2023-05-31,17:44,2023-06-01,18:15
1,ELEANOR ROOSEVELT,IBIZA,IBIZA,2023-05-31 22:28:00,2023-06-01 07:51:00,MUELLE PARAIRES (del 11 al 2),BALEARIA EUROLINEAS MARITIMAS S.A.U.,12.262,2066.0,CHIPRE,123.0,4.0,IBIZA,38.912653,1.442283,2023-05-31,22:28,2023-06-01,07:51
2,TENACIA,BARCELONA,BARCELONA,2023-06-01 04:23:00,2023-06-01 09:59:00,M. 1ª ALIN. PONIENTE SUR (del 14 al 1),TRANSPORTES Y CONSIGNACIONES MARITIMAS BALEAR SAU,25.993,2462.0,ITALIA,199.0,6.0,BARCELONA,41.346387,2.165532,2023-06-01,04:23,2023-06-01,09:59
3,HEDY LAMARR,Valencia,Valencia,2023-06-01 04:53:00,2023-06-01 10:21:00,ALIN.OESTE DE LA PLATAFORMA (del 6 al 1),BALEARIA EUROLINEAS MARITIMAS S.A.U.,26.375,2422.0,CHIPRE,186.0,7.0,Valencia,39.467328,-0.390288,2023-06-01,04:53,2023-06-01,10:21
4,HYPATIA DE ALEJANDRIA,BARCELONA,BARCELONA,2023-06-01 05:19:00,2023-06-01 11:35:00,MUELLE DE LA 2 ALIN. DEL DIQUE DEL OESTE (del ...,BALEARIA EUROLINEAS MARITIMAS S.A.U.,28.658,2181.0,CHIPRE,187.0,7.0,BARCELONA,41.346387,2.165532,2023-06-01,05:19,2023-06-01,11:35


In [35]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [44]:
# Definimos la función
def fetch_openmeteo_data(row):
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": row["LatitudPuertoLlegada"],
        "longitude": row["LongitudPuertoLlegada"],
        "start_date": row["llegada_fecha"],
        "end_date": row["salida_fecha"],
        "daily": ["rain_sum", "wind_speed_10m_max"]
    }
    responses = openmeteo.weather_api(url, params=params)
    
    # Process the response
    response = responses[0]  # Assuming you are interested in the first response
    daily = response.Daily()
    daily_rain_sum = daily.Variables(0).ValuesAsNumpy()
    daily_wind_speed_10m_max = daily.Variables(1).ValuesAsNumpy()

    daily_rain_sum = daily_rain_sum[0]
    daily_wind_speed_10m_max = daily_wind_speed_10m_max[0]
    
    return daily_rain_sum, daily_wind_speed_10m_max

In [45]:
# Apply the function to each row in your DataFrame and store the results in new columns
df[["daily_rain_sum", "daily_wind_speed_10m_max"]] = df.apply(fetch_openmeteo_data, axis=1)

ValueError: Columns must be same length as key

In [19]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"start_date": "2023-10-17",
	"end_date": "2023-10-18",
	"daily": ["rain_sum", "wind_speed_10m_max"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_rain_sum = daily.Variables(0).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(1).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s"),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["rain_sum"] = daily_rain_sum
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

Coordinates 52.5°E 13.400009155273438°N
Elevation 38.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
        date  rain_sum  wind_speed_10m_max
0 2023-10-17       0.0            7.208994
1 2023-10-18       0.0           17.819090


In [22]:
daily_data

{'date': DatetimeIndex(['2023-10-17', '2023-10-18'], dtype='datetime64[ns]', freq='D'),
 'rain_sum': array([0., 0.], dtype=float32),
 'wind_speed_10m_max': array([ 7.208994, 17.81909 ], dtype=float32)}

In [41]:
daily_wind_speed_10m_max[1]

17.81909